In [1]:
import numpy as np
import pandas as pd
import random, copy, os
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool
import multiprocessing

In [17]:
from cvxopt.base import matrix as m
from cvxopt import solvers
from cvxopt.modeling import op, dot, variable, max, min

In [1]:
def set_unique_ids(objects):
    for idx, obj in enumerate(objects):
        obj.set_unique_id(idx)

In [2]:
class Driver:
    def __init__(self, d_id, driver_race, driver_gender, pickup_lat_bin, pickup_long_bin, quota):
        self.d_id = d_id
        self.gender = driver_gender
        self.race = driver_race
        self.latitude = pickup_lat_bin
        self.longitude = pickup_long_bin
        self.quota = quota
        self.is_present = True
    
    def set_unique_id(self, u_id):
        # This is the unique ID used to index the edge matrix. 
        # To find the probability of a driver u accepting a ride of type v, see the entry mat[u.u_id][v.u_id]
        self.u_id = u_id
    
    def __key(self):
#         return (self.gender, self.race, self.latitude, self.longitude)
        return (self.d_id)
    
    def __hash__(self):
        return hash(self.__key())
    
    def __str__(self):
        return "Gender: {}, Race: {}, Lat bin: {}, Long bin: {}, Quota: {}".format(self.gender, self.race,
                                                                                  self.latitude, self.longitude,
                                                                                  self.quota)
        
    def __eq__(self, other):
        if isinstance(other, Driver):
            return self.__key() == other.__key()
        return NotImplemented

class Request:
    def __init__(self, pickup_lat_bin, pickup_long_bin, dropoff_lat_bin, dropoff_long_bin, 
                     requests_gender, requests_race, arrival_rate, distance):
        self.gender = requests_gender
        self.race = requests_race
        self.start_latitude = pickup_lat_bin
        self.start_longitude = pickup_long_bin
        self.end_latitude = dropoff_lat_bin
        self.end_longitude = dropoff_long_bin
        self.arrival_rate = arrival_rate
        self.distance = distance
    
    def set_unique_id(self, u_id):
        # This is the unique ID used to index the edge matrix. 
        # To find the probability of a driver u accepting a ride of type v, see the entry mat[u.u_id][v.u_id]
        self.u_id = u_id

    def __key(self):
        return (self.gender, self.race, self.start_latitude, self.start_longitude, 
                self.end_latitude, self.end_longitude)
    
    def __hash__(self):
        return hash(self.__key())
    
    def __str__(self):
        return "Gender: {}, Race: {}, Start Lat bin: {}, Start Long bin: {}, End Lat bin: {}, End Long bin: {}"\
                .format(self.gender, self.race, self.start_latitude, self.start_longitude, 
                                     self.end_latitude, self.end_longitude)
    
    def __eq__(self, other):
        if isinstance(other, Request):
            return self.__key() == other.__key()
        return NotImplemented

In [2]:
def coordinate_to_index(i, j, probability_matrix):
    """Given a i,j in the probability matrix, convert to the index of the x_f vector"""
    edges_count = np.count_nonzero(probability_matrix[:i,:] != -1) + np.count_nonzero(probability_matrix[i,:j] != -1)
    # count the number of edges of drivers before i and  count the number of edges of ith driver before request j
    return edges_count

def index_to_coordinate(idx, probability_matrix):
    return list(zip(*np.where(probability_matrix != -1)))[idx]

In [14]:
def get_fairness_objective(x_fair, probability_matrix, requests):
    # Minimize the negative of the objective function (same as maximizing the objective function)
    all_requests_fairness = []
    for j in range(probability_matrix.shape[1]):
        mask = [0] * len(x_fair)
        for i in np.where(probability_matrix[:,j] != -1)[0]:
#             print (len(mask), coordinate_to_index(i, j, probability_matrix), i, j, probability_matrix.shape, len(requests))
            mask[coordinate_to_index(i, j, probability_matrix)] = \
                -1 * (probability_matrix[i, j]/requests[j].arrival_rate)
        all_requests_fairness.append(dot(m(mask), x_fair))
    fairness = max(all_requests_fairness) # solver would minimize this
    return fairness

In [ ]:
def get_profit_objective(x_f, probability_matrix, profit_matrix):
    c = []
    for i, j in zip(*np.where(probability_matrix != -1)):
        c.append(-1 * profit_matrix[i,j] * probability_matrix[i,j]) # multiply by -1 since we want to maximise w_f * x_f * p_f but cvxopt minimizes the objective function by default, since minimizing -obj is same as maximizing obj, we multiply our profit objective by a minus sign
    assert len(c) == len(x_f)
    c = m(c)
    profit = dot(c, x_f)
    c, x_f
    return profit, c

In [16]:
def get_inequalities(x, probability_matrix, requests, return_coefficients=False):
    offset = 0
    A, b = [], [] # model all inequalities as A * x <= b
    # models the inequalities 3 and 4 in the writeup
    for i in range(probability_matrix.shape[0]): # iterate over all drivers
        a1, a2 = [0] * len(x), [0] * len(x) # coefficients of inequalities
        edges_count = np.count_nonzero(probability_matrix[i] != -1)
        edges_probabilities = probability_matrix[i][np.where(probability_matrix[i] != -1)]
        assert len(edges_probabilities) == edges_count # sanity check
        a1[offset:offset + edges_count] = edges_probabilities
        a2[offset:offset + edges_count] = [1] * edges_count
        A.append(a1)
        A.append(a2)
        b.append(1)
        b.append(drivers[i].quota)
        offset += edges_count
    # Models the inequality -1 * x_f <= 0 for all edges
    for i in range(len(x)):
        a1 = [0] * len(x)
        a1[i] = -1
        A.append(a1)
        b.append(0)
    # Models inequality 5 in the writeup
    for j in range(probability_matrix.shape[1]):# iterate over all request types
        # j -> request; i-> driver
        a1 = [0] * len(x)
        for i in np.where(probability_matrix[:,j] != -1)[0]:
            a1[coordinate_to_index(i, j, probability_matrix)] = 1
        A.append(a1)
        b.append(requests[j].arrival_rate)
    print (len(A), len(b), len(A[0]), len(x))

    A, b = m(A).T, m(b)

    if not return_coefficients:
        print (type(A*x))
        inequality = (A * x <= b)
        return inequality
    else:
        return A, b

In [5]:
def lp_solution_sanity_check(x_f, x_fair, probability_matrix, requests):
    # Sanity Check: sum of all x_f for a request should be less than r_v
    for j in range(probability_matrix.shape[1]):
        sum_x_f = 0
        for i in np.where(probability_matrix[:,j] != -1)[0]:
            sum_x_f += x_f.value[coordinate_to_index(i, j, probability_matrix)]
        if sum_x_f/requests[j].arrival_rate > 1:
            assert np.isclose(sum_x_f/requests[j].arrival_rate, 1, atol=0.000001, rtol=0)
    #     print (min(x_f.value/r.arrival_rate), max(x_f.value/r.arrival_rate))
    #     print (min(x_fair.value/r.arrival_rate), max(x_fair.value/r.arrival_rate))
    for i in range(len(x_f)):
        if x_f.value[i] < 0:
            assert np.isclose(x_f.value[i], 0, atol=0.000001, rtol=0)
            x_f.value[i] = 0.0
        if x_fair.value[i] < 0:
            assert np.isclose(x_fair.value[i], 0, atol=0.000001, rtol=0)
            x_fair.value[i] = 0.0

In [7]:
def measure_exact_fairness(all_requests, matching):
    fairness_measure = []
    requests = list(set(all_requests))
    for r in requests:
        fairness_measure.append(
            np.count_nonzero(np.array(matching)[np.array(all_requests) == r] != None)/r.arrival_rate
        )
    fairness_measure = np.array(fairness_measure)
    return np.min(fairness_measure[fairness_measure > 0]) if len(fairness_measure[fairness_measure > 0]) > 0 else 0

def measure_fairness_edges_count(all_requests, matching, requests):
    expected_edge_traversals = []
    for r in requests: # note that requests here ensures that edge counts are ordered consistently
        expected_edge_traversals.append(
            np.count_nonzero(np.array(matching)[np.array(all_requests) == r] != None)
        )
    return np.array(expected_edge_traversals)
    
def measure_expected_fairness(r, x_f, y_f, alpha, beta, probability_matrix):
    """
    r is a request type, x_f is the optimal profit assignment, y_f is the optimal fair assignment, 
    alpha is the weight to be given to profitable assignment and beta is the weight to be given to the fair assignment
    """
    expectation = 0
    for d in np.where(probability_matrix[:,r.u_id] != -1)[0]:
        idx = coordinate_to_index(d, r.u_id, probability_matrix)
        expectation += probability_matrix[d, r.u_id] * (alpha * x_f.value[idx] + beta * y_f.value[idx])
    expectation /= r.arrival_rate
    return expectation

def measure_expected_profit(r, x_f, y_f, alpha, beta, probability_matrix):
    """
    r is a request type, x_f is the optimal profit assignment, y_f is the optimal fair assignment, 
    alpha is the weight to be given to profitable assignment and beta is the weight to be given to the fair assignment
    """
    expectation = 0
    for d in np.where(probability_matrix[:,r.u_id] != -1)[0]:
        idx = coordinate_to_index(d, r.u_id, probability_matrix)
        expectation += probability_matrix[d, r.u_id] * (alpha * x_f.value[idx] + beta * y_f.value[idx]) * r.distance
    return expectation

In [2]:
def driver_acceptance(driver, request, probability_matrix):
    p_f = probability_matrix[driver.u_id, request.u_id]
#     print ("Driver acceptance prob: {}, request: {}, is_present: {}".format(p_f, request.u_id, driver.is_present))
    if driver.is_present:
        if driver.quota == 0: # Driver HAS to accept the request
            driver.is_present = False
            return driver
        decision = np.random.choice(np.arange(2), size=1, p=[p_f, 1-p_f])[0]
        if decision == 0: #driver accepted the trip
#             print ("Driver accepted request {}!".format(request.u_id))
            driver.is_present = False
            return driver
        else:
            driver.quota -= 1
            return None
    else:
        return None
    
def NAdap(alpha, beta, request, drivers_copy, probability_matrix, x_f, x_fair):
    # There are 2 possible actions, choose assignment based on x_f* (profitable) and y_f* (fair) or reject
    action = np.random.choice(np.arange(2), size=1, p=[alpha + beta, 1 - alpha - beta] if alpha + beta < 1 \
                              and alpha + beta > 0 else [int(alpha + beta), int(1 - alpha - beta)])[0]
    if action == 0: # choose to assign
        edge_coordinates, edge_indices, sample_probabilities_edge = [], [], []
        for i in np.where(probability_matrix[:,request.u_id] != -1)[0]:
            edge_coordinates.append((i, request.u_id))
            idx = coordinate_to_index(i, request.u_id, probability_matrix)
            edge_indices.append(idx)
            sample_probabilities_edge.append(alpha * x_f.value[idx]/request.arrival_rate + \
                                         beta * x_fair.value[idx]/request.arrival_rate)
#         print ("Edges: {}, Sample edge probabilities: {}".format(edge_coordinates, sample_probabilities_edge))
        # difference between the sum of edge probs and 1
        difference = 1 - np.sum(sample_probabilities_edge)
        # scale up probabilities to ensure they add up to one
        # only scale up the non-zero probabilities!!
        if np.isclose(difference, 0, atol=0.000001, rtol=0):
            sample_probabilities_edge[np.argmax(sample_probabilities_edge)] += difference
        else:
            indices_to_scale = np.where(np.array(sample_probabilities_edge) != 0)[0]
            if len(indices_to_scale) == 0:
                sample_probabilities_edge = \
                    [x + difference/len(sample_probabilities_edge) for x in sample_probabilities_edge]
            else:
#                 print ("Here!")
                for idx in indices_to_scale:
                    sample_probabilities_edge[idx] += difference/len(indices_to_scale)
        try:
            sampled_edge = np.random.choice(np.array(edge_indices), size=1, p=sample_probabilities_edge)[0]
        except:
            print (sample_probabilities_edge)
        driver = drivers_copy[edge_coordinates[edge_indices.index(sampled_edge)][0]]
        return driver_acceptance(driver, request, probability_matrix)
    else: # reject the request
        assert alpha + beta < 1, "If alpha + beta == 1, this should not happen"
        return None

def run_algorithm(all_requests, drivers_copy, probability_matrix, x_f, x_fair, alpha=0.5, beta=0.5):
    exact_profit, count, available_drivers = 0, 0, count_available_drivers(drivers_copy)
    assert available_drivers == len(drivers_copy)
    matches = []
    random.shuffle(all_requests)
    for r in all_requests:
        matched_driver = NAdap(alpha, beta, r, drivers_copy, probability_matrix, x_f, x_fair)
        matches.append(matched_driver)
        if matched_driver is not None:
#             print ("Driver found! : Driver: {}, Request: {}".format(matched_driver.u_id, r.u_id))
            available_drivers -= 1
            assert available_drivers == count_available_drivers(drivers_copy)
            exact_profit += r.distance
            count += 1
    return exact_profit, count, matches

def count_available_drivers(drivers_copy):
    count = 0
    for d in drivers_copy:
        if d.is_present:
            count += 1
    return count

In [10]:
def run_greedy(all_requests, drivers_copy, probability_matrix, profit_matrix):
    matches, profit = [], 0
    for request in all_requests:
        available_drivers, profits = [], []
        for idx in np.where(probability_matrix[:,request.u_id] != -1)[0]:
            assert drivers_copy[idx].u_id == idx
            if drivers_copy[idx].is_present:
                available_drivers.append(drivers_copy[idx])
                assert probability_matrix[idx, request.u_id] != -1
                profits.append(profit_matrix[idx, request.u_id])
        if len(available_drivers) == 0:
            assigned_driver = None
        else:
            assigned_driver = driver_acceptance(available_drivers[np.argmax(profits)], 
                                                    request, probability_matrix)
        matches.append(assigned_driver)
        if assigned_driver is not None:
            profit += request.distance
    return matches, profit

def run_uniform(all_requests, drivers_copy, probability_matrix):
    matches, profit = [], 0
    for r in all_requests:
        driver_idx = np.random.choice(np.where(probability_matrix[:,r.u_id] != -1)[0], size=1)[0]
        assert drivers_copy[driver_idx].u_id == driver_idx
        assert probability_matrix[driver_idx, r.u_id] != -1
        assigned_driver = driver_acceptance(drivers_copy[driver_idx], r, probability_matrix)
        matches.append(assigned_driver)
        if assigned_driver is not None:
            profit += r.distance
    return matches, profit

In [8]:
def calculate_fairness_from_array(edges_count, num_loops, requests):
    fairness_counts_all_runs = np.array(edges_count)
#     error_counts = np.min(fairness_counts, axis=1) # will be used to calculate fairness for each run
#     print (error_counts)
#     error_args = np.argmin(fairness_counts, axis=1)
#     assert len(error_counts) == num_loops and len(error_args) == num_loops
#     for i in range(len(error_args)):
#         error_counts[i] /= requests[error_args[i]].arrival_rate
    fairness_counts = np.sum(fairness_counts_all_runs, axis=0)/num_loops
    for idx in range(len(requests)):
        fairness_counts[idx] /= requests[idx].arrival_rate
    
#     print(requests[np.argmin(fairness_counts)].arrival_rate)
#     fairness_error = np.std(
#         fairness_counts_all_runs[:,np.argmin(fairness_counts)] / \
#         requests[np.argmin(fairness_counts)].arrival_rate)
#     print (fairness_error)
    
    return np.min(fairness_counts)

In [ ]:
def get_matching_results(all_requests, drivers, probability_matrix, x_f, x_fair, alphas, num_loops):
    algorithm_params = []
    for alpha in alphas:
        for i in range(num_loops):
            algorithm_params.append([all_requests, [copy.deepcopy(d) for d in drivers], 
                                     probability_matrix, x_f, x_fair, alpha, 1-alpha])

    with Pool(multiprocessing.cpu_count()) as p:
        matching_results = p.starmap(run_algorithm, algorithm_params)
    return matching_results

In [ ]:
def get_edges_count_results(all_requests, matching_results, requests):
    fairness_measure_params = []
    for matching_result in matching_results:
        fairness_measure_params.append([all_requests, matching_result[2], requests])

    with Pool(multiprocessing.cpu_count()) as p:
        edges_count_results = p.starmap(measure_fairness_edges_count, fairness_measure_params)
    return edges_count_results

In [ ]:
def get_profit_fairness_crs(matching_results, edges_count_results, num_loops, 
                            requests, alphas, optimal_profit, optimal_fairness):
    profit_crs, profit_errors, fairness_crs, fairness_errors = [], [], [], []
    for j in range(len(alphas)):
        expected_profit, std_dev_profit = 0, []
        for i in range(num_loops):
            expected_profit += matching_results[j * num_loops + i][0]
            std_dev_profit.append(matching_results[j * num_loops + i][0])
        expected_profit /= num_loops
        std_dev_profit = np.std(std_dev_profit)
        profit_crs.append(expected_profit/optimal_profit)
        profit_errors.append(std_dev_profit/optimal_profit)

        fairness_measure = calculate_fairness_from_array(
            edges_count_results[j*num_loops:(j+1)*num_loops], num_loops, requests)
        print (expected_profit/optimal_profit, fairness_measure/optimal_fairness)
        fairness_std_dev = 0
        fairness_crs.append(fairness_measure/optimal_fairness)
        fairness_errors.append(fairness_std_dev/optimal_fairness)
    return profit_crs, profit_errors, fairness_crs, fairness_errors

In [ ]:
def calculate_T(requests):
    return np.sum([r.arrival_rate for r in requests])